In [4]:
# What version of Python do you have?
import sys
import os
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import re
import mne
import pathlib
import openpyxl
from datetime import datetime
import scipy
import pytz
import random
import gc
import os
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Sequential
from matplotlib import pyplot as plt
pd.io.parquet.get_engine('auto').__class__
%matplotlib inline

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.7.0
Keras Version: 2.7.0

Python 3.9.7 (default, Sep 16 2021, 23:53:23) 
[Clang 12.0.0 ]
Pandas 1.3.4
Scikit-Learn 1.0.1
GPU is available


Globals

In [2]:
cwd = os.getcwd()
FREQ = 256
database_path = 'Dataset/CHB-MIT/chb-mit-scalp-eeg-database-1.0.0/'
filtered_database_path = '../Dataset/CHB-MIT/Filtered-chb-mit/'
filted_db_parquet_path = "Dataset/CHB-MIT/dataframe-parquet"
save_csv_path = "../DataSet/Eks_DB/filtered_df_csv"
edf_file_type = ".edf"
patient_one_path = 'chb04/'
summary_txt_file_type = "-summary.txt"
external_hardisk_drive_path = os.path.dirname('/Volumes/LaCie/Database/')
cwd

'/Users/niklashjort/Desktop/Notes/Speciale/projects/DataHandling/MIT_CHB/jupyter'

In [3]:
external_hardisk_drive_path = os.path.dirname('/Volumes/NHR HDD/Eks_DB/')
filtered_database_path = external_hardisk_drive_path + "/MIT/Filtered-chb-mit/"
save_df_csv_path = external_hardisk_drive_path + "/MIT/Filtered_csv/"
summary_txt_file_type = "-summary.txt"
edf_file_type = ".edf"

Get file paths and info.txt

In [5]:
def get_all_patient_folder_names(database_path):
    folders = os.listdir(database_path)
    patient_folder_names = [(database_path + "/" + x) for x in folders if (x.find(".DS_Store") == -1) if not "._" in x]

    return patient_folder_names

def get_all_file_names(directory):
    files = os.listdir(directory)
    edfFileNameList = [(directory + "/" + x) for x in files if (x.endswith(edf_file_type)) if not "._" in x]
    summary_info_file_name = [(directory + "/" + x) for x in files if (x.endswith(summary_txt_file_type))]
    return (summary_info_file_name[0], edfFileNameList)

In [6]:
get_all_patient_folder_names(filtered_database_path)

['/Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit//chb20',
 '/Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit//chb18',
 '/Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit//chb19',
 '/Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit//chb21',
 '/Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit//chb17',
 '/Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit//chb10',
 '/Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit//chb03',
 '/Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit//chb02',
 '/Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit//chb22',
 '/Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit//chb07',
 '/Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit//chb01']

In [7]:
x, y = get_all_file_names("/Volumes/NHR HDD/CHB-MIT/Filtered-chb-mit//chb20")
print(y[0:5])

['/Volumes/NHR HDD/CHB-MIT/Filtered-chb-mit//chb20/chb20_59.edf', '/Volumes/NHR HDD/CHB-MIT/Filtered-chb-mit//chb20/chb20_60.edf', '/Volumes/NHR HDD/CHB-MIT/Filtered-chb-mit//chb20/chb20_04.edf', '/Volumes/NHR HDD/CHB-MIT/Filtered-chb-mit//chb20/chb20_11.edf', '/Volumes/NHR HDD/CHB-MIT/Filtered-chb-mit//chb20/chb20_05.edf']


Read EDF

In [8]:
def ReadEdfFile(FileName, print_reader_info = False):
    if(print_reader_info):
        data = mne.io.read_raw_edf(FileName)
        raw_data = data.get_data()
        converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
        print(data.info)
        print(data.ch_names)
        return converted_raw
    else:
        data = mne.io.read_raw_edf(FileName, verbose='error')
        raw_data = data.get_data()
        converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
        return converted_raw


In [9]:
x = mne.io.read_raw_edf(filtered_database_path + "chb01/chb01_06.edf")
y = ReadEdfFile(filtered_database_path + "chb10/chb10_12.edf", print_reader_info=True)


Extracting EDF parameters from /Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit/chb01/chb01_06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Volumes/NHR HDD/Eks_DB/MIT/Filtered-chb-mit/chb10/chb10_12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/var/folders/0v/m6wt8rqj7s1dcljdyjrdfxmw0000gn/T/ipykernel_11668/2938848453.py:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  x = mne.io.read_raw_edf(filtered_database_path + "chb01/chb01_06.edf")
/var/folders/0v/m6wt8rqj7s1dcljdyjrdfxmw0000gn/T/ipykernel_11668/3376001831.py:3: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(FileName)


<Info | 7 non-empty values
 bads: []
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2051-11-23 15:55:00 UTC
 nchan: 23
 projs: []
 sfreq: 256.0 Hz
>
['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']


In [10]:
# print(test_df.info(memory_usage='deep'))
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [11]:
def downcast_dtypes(df):
    _start = df.memory_usage(deep=True).sum() / 1024 ** 2
    float_cols = [c for c in df if df[c].dtype == 'float64']
    int_cols = [c for c in df if df[c].dtype in ['int64', 'int32']]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    _end = df.memory_usage(deep=True).sum() / 1024 ** 2
    saved_time = (_start - _end) / _start * 100
    #print(f"Saved: {saved_time:.2f}%")
    return df

format summary txt file

In [12]:
def read_format_info_file(txt_summary_file_path):
    str_container = ""
    with open(txt_summary_file_path, 'r') as f:
        for line in f:
            str_container += str(line).replace("\n", "<br>")

    formatted_str = re.findall('(.*?)<br><br>', str_container)
    bla = [x.group() for x in re.finditer('(.*?)<br><br>', str_container)]

    formatted_str = [x for x in formatted_str if "File Name" in x]

    return formatted_str

In [13]:
new_str = read_format_info_file(filtered_database_path + "chb01/chb01-summary.txt")

In [14]:
new_str[0:5]

['File Name: chb01_01.edf<br>File Start Time: 11:42:54<br>File End Time: 12:42:54<br>Number of Seizures in File: 0',
 'File Name: chb01_02.edf<br>File Start Time: 12:42:57<br>File End Time: 13:42:57<br>Number of Seizures in File: 0',
 'File Name: chb01_03.edf<br>File Start Time: 13:43:04<br>File End Time: 14:43:04<br>Number of Seizures in File: 1<br>Seizure Start Time: 2996 seconds<br>Seizure End Time: 3036 seconds',
 'File Name: chb01_04.edf<br>File Start Time: 14:43:12<br>File End Time: 15:43:12<br>Number of Seizures in File: 1<br>Seizure Start Time: 1467 seconds<br>Seizure End Time: 1494 seconds',
 'File Name: chb01_05.edf<br>File Start Time: 15:43:19<br>File End Time: 16:43:19<br>Number of Seizures in File: 0']

In [15]:
class FileInformationContainer:
    def __init__(self, information_str):
        self.information_str = self.clean_string(information_str)
        self.file_name = self.set_filename()
        self.time_start = self.set_file_time_start_ms()
        self.sz_info = self.set_sz_information()
        
    def clean_string(self, uncleaned_str):
        return uncleaned_str.replace("<br>", " ")

    def set_filename(self):
        filename_found = re.match(r"^File Name: (.+?).edf", self.information_str)
        if filename_found:
            return filename_found.group(1)
        else:
            print(f"{self.file_name} failed get_filename")
            return "filename not found"
    
    def get_milli_sec(self, time_str):
        """Get Seconds from time."""
        dt_obj = datetime.strptime(time_str,'%H:%M:%S')
        millisec = dt_obj.timestamp() * 1000
        return millisec

    def set_file_time_start_ms(self):
        time_start_found = re.match(r".*File Start Time: (.*?) File", self.information_str)
        if time_start_found:
            try:
                return self.get_milli_sec(time_start_found.group(1))
            except Exception as e :
                print(f"{self.file_name}: error {e} cannot convert to ms time")
                return f"{e}"
        else:
            print(f"{self.file_name} failed get_file_time_start_ms")
            return "time start not found"
    
    def get_sz_count(self):
        sz_count = 0
        count_found = re.search(r".*Seizures in File: (.*?) Seizure", self.information_str)
        if count_found:
            sz_count = count_found.group(1)
        if int(sz_count) > 0:
            return int(sz_count)
        else:
            return 0
        

    def set_sz_information(self):
        if(type(self.get_sz_count()) != None and self.get_sz_count() > 0):
            try:
                pattern = re.compile(r"Seizure [1-9] (?P<state>[?:Start|End]+) Time: (?P<Sec>[0-9]+)")
                myList = [m.groupdict() for m in pattern.finditer(self.information_str)]
                if(len(myList) <= 0):
                     pattern = re.compile(r"Seizure (?P<state>[?:Start|End]+) Time: (?P<Sec>[0-9]+)")
                     myList = [m.groupdict() for m in pattern.finditer(self.information_str)]
                for item in myList:
                    converted_time = int(item.get("Sec"))
                    item["Sec"] = converted_time
                formatted = []
                for i in range(0, len(myList), 2):
                    formatted.append({"sz_start" : myList[i]["Sec"], "sz_end" : myList[i+1]["Sec"]})
                return formatted
            except Exception as e:
                print(f"set_sz_information failed at file: {self.file_name} with the following exception: {e}")
        else:
            return []

    def get(self):
        return self.information_str

Timestamp inserts

In [16]:
def insert_time_stamp(dataframe, file_start_time):
    period_row_increment_value =  (1 / 256) * 1000
    dataframe.insert(0, "timestamp", [file_start_time + i * period_row_increment_value for i in dataframe.index])

Insert classes

In [17]:
def df_save_compress(filename, df):
    print("filename " + filename)
    df.to_csv(f"{filename}.csv")

In [18]:
def nanpow2db(y):
    if isinstance(y, int) or isinstance(y, float):
        if y == 0:
            return np.nan
        else:
            ydB = 10 * np.log10(y)
    else:
        if isinstance(y, list):  # if list, turn into array
            y = np.asarray(y)
        y = y.astype(float)  # make sure it's a float array so we can put nans in it
        y[y == 0] = np.nan
        ydB = 10 * np.log10(y)

    return ydB

In [19]:
def find_log_min_max_welch(channel, signal,  filename, log_file, freq=500):
    '''
    Calculate Power median of the signal.
    Mask the PSD if filters creates low power.
    Then log it to txt and use for pcolormesh normalization.
    '''

    # PSD w. Welch
    f, Pxx = scipy.signal.welch(signal[channel], fs=freq, window='hanning', scaling='density', average='median', detrend=False)
    Pxx_den_db = nanpow2db(Pxx)

    # Creating and remove frequencies === False e.g. not in mask

    # mask for low pass filter
    mask = (f < 240)
    m_Pxx_den_db = Pxx_den_db[mask]
    m_f = f[mask]

    # mask for bandpass
    mask = (m_f < 47) | (m_f > 53)
    m_Pxx_den_db = m_Pxx_den_db[mask]
    m_f = m_f[mask]
    
    # mask for bandpass
    mask = (m_f < 97) | (m_f > 103)
    m_Pxx_den_db = m_Pxx_den_db[mask]
    m_f = m_f[mask]

    # mask for highpass
    mask = (m_f > 1)
    m_Pxx_den_db = m_Pxx_den_db[mask]
    m_f = m_f[mask]

    # Debugging only:
    # plt.plot(m_f, m_Pxx_den_db)
    # plt.savefig("hi.png")

    # Filtered and masked global median min and max value
    mn = np.min(m_Pxx_den_db)
    mx = np.max(m_Pxx_den_db)

    # Log file name and value pair
    file_object = open(log_file, "a")
    file_object.write(f"\nfilename: {filename} \nchannel: {channel} \nmin: {mn} \nmax: {mx} \n")
    file_object.close()

    # Garbage collection
    del f, Pxx, mn, mx, mask, m_Pxx_den_db, m_f
    gc.collect()

In [25]:
from scipy import signal


def butter_highpass_filter(data, cutoff, fs, order=6):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

def butter_lowpass_filter(data, cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

def butter_bandstop_filter(data, lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    i, u = signal.butter(order, [low, high], btype='bandstop')
    y = signal.lfilter(i, u, data)
    return y

def butter_bandpass_filter(data, lowcut, highcut, fs, order=1):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    y = signal.filtfilt(b, a, data)
    return y


def apply_filter(series, FREQ, low=False):
    series = butter_bandstop_filter(series, 97, 103, FREQ, order=4)
    series = butter_bandstop_filter(series, 47, 53, FREQ, order=4)
    series = butter_highpass_filter(series, 1, FREQ, order=4)
    if low:
        series = butter_lowpass_filter(series, 125, FREQ, order=3)
    return series


def apply_ecg_filter(series, FREQ):
    series = butter_bandstop_filter(series, 47, 53, FREQ, order=4)
    series = butter_highpass_filter(series, 1, FREQ, order=4)
    series = butter_bandpass_filter(series, 1, 98, FREQ)
    return series

In [27]:
class_mapping = {"Seizure": 1, "Preictal": 2, "Interictal": 3}

def insert_class_col(dataframe_copy, sz_info_list, save_name):
    print(f"modtaget string: {sz_info_list}")

    save_folder = ""

    channel_to_save = ['FP1-F7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4']

    dataframe = dataframe_copy[channel_to_save]

    for col in dataframe.columns:
        dataframe[col] = apply_filter(dataframe[col], FREQ, low=True)
        find_log_min_max_welch(col, dataframe, save_name, f"{save_df_csv_path}/welch_info.txt")

    quarter_ensurance = 15 * 60 * 256
    
    if "class" not in dataframe.columns:
        dataframe.insert(0, "class", np.nan)

    print(dataframe.columns)

    if len(sz_info_list) == 0:
        int = dataframe[0:-quarter_ensurance]
        df_save_compress(save_df_csv_path + "Interictal/" + f"{save_name}_none_sz", int)
        print(f"NO SZ len int {len(int)}")

    else:
        for item in sz_info_list:
            
            sz_start = item["sz_start"] * 256
            sz_end = item["sz_end"] * 256
            print(f"sz_start index = {sz_start}")
            print(f"sz_end: {sz_end}")
            preictal_start = sz_start - (15 * 60 * 256) if (sz_start - (15 * 60 * 256)) >= 0 else 0
            print(f"prei start {preictal_start}")
            interictal_start = preictal_start - (60 * 60 * 256) if preictal_start > 0 and preictal_start - (60 * 60 * 256) >= 0 else 0
            #print(f"dur: {sz_end - sz_start}")

            dataframe.loc[(dataframe.index > sz_start) & (dataframe.index < sz_end), 'class'] = class_mapping['Seizure']
            dataframe.loc[(dataframe.index > preictal_start) & (dataframe.index < sz_start), 'class'] = class_mapping['Preictal']
            dataframe.loc[(dataframe.index > interictal_start) & (dataframe.index < preictal_start), 'class'] = class_mapping['Interictal']
            # dataframe['class'][sz_start : sz_end] = class_mapping["Seizure"]
            # dataframe['class'][preictal_start : sz_start] = class_mapping["Preictal"]
            # dataframe['class'][interictal_start : preictal_start] = class_mapping["Interictal"]
            print(dataframe['class'].value_counts())

        for item in sz_info_list:
            
            sz_start = item["sz_start"] * 256
            sz_end = item["sz_end"] * 256
            print(f"sz_start index = {sz_start}")
            print(f"sz_end: {sz_end}")
            preictal_start = sz_start - (15 * 60 * 256) if (sz_start - (15 * 60 * 256)) >= 0 else 0
            print(f"prei start {preictal_start}")
            interictal_start = preictal_start - (60 * 60 * 256) if preictal_start > 0 and preictal_start - (60 * 60 * 256) >= 0 else 0
            #print(f"dur: {sz_end - sz_start}")

            prei = dataframe[(dataframe.index >= preictal_start) & (dataframe.index < sz_start) & (dataframe['class'] != class_mapping["Seizure"])].copy()
            prei = dataframe[preictal_start:sz_start]
            df_save_compress(save_df_csv_path + "Preictal/" + f"{save_name}_prei_{item['sz_start']}", prei)
            print(f"prei len {len(prei)}")

            sz = dataframe[(dataframe.index >= sz_start) & (dataframe.index < sz_end)].copy()
            df_save_compress(save_df_csv_path + "Seizure/" + f"{save_name}_sz_{item['sz_start']}", sz)
            print(f"sz len {len(sz)}")

            inte = dataframe[(dataframe.index >= interictal_start) & (dataframe.index < preictal_start) & (dataframe['class'] != class_mapping["Seizure"]) & (dataframe['class'] != class_mapping["Preictal"])].copy()
            if len(inte) > 1*60*256:
                df_save_compress(save_df_csv_path+ "Interictal/" + f"{save_name}_int_{item['sz_start']}", inte)
            print(f"int len: {len(inte)}")

Compress and save

In [23]:
def sort_remove_files(list_obj):
  new_lst = []
  for x in range(0, len(list_obj)):
    if x+1 < len(list_obj) and FileInformationContainer(info_txt[x + 1]).get_sz_count() > 0 and FileInformationContainer(info_txt[x]).get_sz_count() == 0:
        new_lst.append(list_obj[x])
    if FileInformationContainer(list_obj[x]).get_sz_count() > 0:
      new_lst.append(list_obj[x])
  return new_lst




Run script

In [28]:
pat_folders = get_all_patient_folder_names(filtered_database_path)

for patient in pat_folders:
    #print(patient)
    current_patient = patient
    info_txt_path, edf_files = get_all_file_names(current_patient)
    # read & extract information
    info_txt = read_format_info_file(info_txt_path)
    sort_info_txt = sort_remove_files(info_txt)
    for line in sort_info_txt:
        print(f"l: {line}")
        edf_info_container = FileInformationContainer(line)
       # print(f"EDF_CONTAINER: info_string passed {edf_info_container.information_str}")
        #print(f"EDF_CONTAINER: ts_start {edf_info_container.time_start}")
       # print(f"EDF_CONTAINER: sz_info {edf_info_container.sz_info}")
        selected_edf_path = [x for x in edf_files if (edf_info_container.file_name in x)][0]
        edf_df = ReadEdfFile(selected_edf_path)
        if edf_df is not None:
            #edf_df = downcast_dtypes(edf_df)
            #insert_time_stamp(edf_df, edf_info_container.time_start)
            #print(f"info list = {edf_info_container.sz_info}")
            insert_class_col(edf_df, edf_info_container.sz_info, edf_info_container.file_name)
            #print(f"filename: {edf_info_container.file_name} classes: {edf_df['class'].value_counts()}")
            print("-------------------------------------------------------")
            #print(edf_info_container.file_name)
            #df_save_compress(edf_info_container.file_name, edf_df)
            #print(f"saved: {edf_info_container.file_name}")

l: File Name: chb20_11.edf<br>File Start Time: 02:56:46<br>File End Time: 3:56:46<br>Number of Seizures in File: 0
modtaget string: []


/var/folders/0v/m6wt8rqj7s1dcljdyjrdfxmw0000gn/T/ipykernel_11668/743716675.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col] = apply_filter(dataframe[col], FREQ, low=True)


Index(['class', 'FP1-F7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1',
       'FP2-F4', 'F4-C4'],
      dtype='object')
filename /Volumes/NHR HDD/Eks_DB/MIT/Filtered_csv/Interictal/chb20_11_none_sz
NO SZ len int 691200
-------------------------------------------------------
l: File Name: chb20_12.edf<br>File Start Time: 03:56:53<br>File End Time: 4:56:53<br>Number of Seizures in File: 1<br>Seizure 1 Start Time: 94 seconds<br>Seizure 1 End Time: 123 seconds
modtaget string: [{'sz_start': 94, 'sz_end': 123}]
Index(['class', 'FP1-F7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1',
       'FP2-F4', 'F4-C4'],
      dtype='object')
sz_start index = 24064
sz_end: 31488
prei start 0
2.0    24063
1.0     7423
Name: class, dtype: int64
sz_start index = 24064
sz_end: 31488
prei start 0
filename /Volumes/NHR HDD/Eks_DB/MIT/Filtered_csv/Preictal/chb20_12_prei_94


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


prei len 24064
filename /Volumes/NHR HDD/Eks_DB/MIT/Filtered_csv/Seizure/chb20_12_sz_94
sz len 7424
int len: 0
-------------------------------------------------------
l: File Name: chb20_13.edf<br>File Start Time: 04:57:00<br>File End Time: 5:57:00<br>Number of Seizures in File: 2<br>Seizure 1 Start Time: 1440 seconds<br>Seizure 1 End Time: 1470 seconds<br>Seizure 2 Start Time: 2498 seconds<br>Seizure 2 End Time: 2537 seconds
modtaget string: [{'sz_start': 1440, 'sz_end': 1470}, {'sz_start': 2498, 'sz_end': 2537}]
Index(['class', 'FP1-F7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1',
       'FP2-F4', 'F4-C4'],
      dtype='object')
sz_start index = 368640
sz_end: 376320
prei start 138240
2.0    230399
3.0    138239
1.0      7679
Name: class, dtype: int64
sz_start index = 639488
sz_end: 649472
prei start 409088
3.0    409087
2.0    230399
1.0      9983
Name: class, dtype: int64
sz_start index = 368640
sz_end: 376320
prei start 138240
filename /Volumes/NHR HDD/Eks_DB/MIT/Filtered_csv/Pr